# Linear Regrssion Model to Predict Service Times 

In [ ]:
!pip install imblearn

     |████████████████████████████████| 167 kB 16.4 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import *
from sklearn.metrics         import *
from imblearn.pipeline import Pipeline
from sklearn.preprocessing   import StandardScaler
from sklearn.preprocessing   import OneHotEncoder
from sklearn.preprocessing   import LabelEncoder
from   sklearn.compose            import *
from   sklearn.ensemble           import RandomForestClassifier, ExtraTreesClassifier, IsolationForest
from   sklearn.experimental       import enable_iterative_imputer
from   sklearn.impute             import *
from   sklearn.linear_model       import LogisticRegression, PassiveAggressiveClassifier, RidgeClassifier, SGDClassifier
from   sklearn.metrics            import roc_auc_score # We have not covered it yet in class. The basics - AUC is from 0 to 1 and higher is better.
#from   sklearn.pipeline           import Pipeline
from   sklearn.preprocessing      import *
from   sklearn.tree               import *
from   sklearn.metrics            import balanced_accuracy_score
from   sklearn.feature_selection import *

In [ ]:
from imblearn.over_sampling import *

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.covariance import *

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
rgg_rss = pd.read_excel('RGG_RSS_final.xlsx')
Neighborhood=rgg_rss['Neighborhood'].unique()

In [ ]:
rgg_rss.columns

Index(['Commodity', 'Vehicle Type', 'Inside/Curb', '16 gal', '20 gal',
       '32 gal', '64 gal', '96 gal', 'Meandor', 'Route', 'Tipper', 'Day',
       '#Units', 'Time', '1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
       '5 yrd', '6 yrd', 'Company', 'Outlier', 'Truck', 'Neighborhood',
       'Address_Street', 'Code', 'Has Key'],
      dtype='object')

In [ ]:
X=rgg_rss[['Commodity', '16 gal', '20 gal', '32 gal', '64 gal', '96 gal','1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
       '5 yrd', '6 yrd', '#Units',
       'Company', 'Has Key']]
y=rgg_rss['Time']

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12826 entries, 0 to 12825
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Commodity  12826 non-null  object
 1   16 gal     12826 non-null  int64 
 2   20 gal     12826 non-null  int64 
 3   32 gal     12826 non-null  int64 
 4   64 gal     12826 non-null  int64 
 5   96 gal     12826 non-null  int64 
 6   1 yrd      12826 non-null  object
 7   1.5 yrd    12826 non-null  int64 
 8   2 yrd      12826 non-null  int64 
 9   3 yrd      12826 non-null  int64 
 10  4 yrd      12826 non-null  int64 
 11  5 yrd      12826 non-null  int64 
 12  6 yrd      12826 non-null  int64 
 13  #Units     12826 non-null  int64 
 14  Company    12826 non-null  object
 15  Has Key    12826 non-null  bool  
dtypes: bool(1), int64(12), object(3)
memory usage: 1.5+ MB


In [ ]:
X=rgg_rss[['Commodity', '16 gal', '20 gal', '32 gal', '64 gal', '96 gal','1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
       '5 yrd', '6 yrd', '#Units',
       'Company', 'Has Key']]
y=rgg_rss['Time']

X=X.replace('na',np.nan)
X=X.replace('nan',np.nan)
X=X.replace('.',np.nan)

preprocessing = ColumnTransformer([
                            ('scaler',StandardScaler(),[1,2,3,4,5,6,7,8,9,10,11,12,13]),                       
                            ('categorical', OneHotEncoder(handle_unknown='ignore'), [0,14,15])
                            ])

pipe = Pipeline([
                #('col_selection',FunctionTransformer(categorical_col_transform,validate=False)),
                ('imputation',SimpleImputer(missing_values=np.nan,strategy="most_frequent")),
                ('preprocessing',preprocessing),
                ('Isol',IsolationForest(contamination=0.1))]) 


yhat = pipe.fit_predict(X)

mask = yhat != -1
X, y = X[mask], y[mask]


In [ ]:
len(mask)

12826

In [ ]:
final_data=rgg_rss[mask]

In [ ]:

#final_data=final_data.loc[final_data['Time']>15]
final_data=final_data.replace('na',np.nan)
final_data=final_data.replace('nan',np.nan)
final_data=final_data.replace('.',np.nan)
final_data.sort_values(by=['Time','1 yrd'],ascending=False)

,Commodity,Vehicle Type,Inside/Curb,16 gal,20 gal,32 gal,64 gal,96 gal,Meandor,Route,...,4 yrd,5 yrd,6 yrd,Company,Outlier,Truck,Neighborhood,Address_Street,Code,Has Key
9787,Recycle,R-HEIL,I,0,0,0,1,5,Yes,981,...,0,0,0,RGG,1,RL,Nob Hill,1100_arkin,NaN,False
9786,Recycle,R-HEIL,I,0,0,0,0,7,No,981,...,0,0,0,RGG,1,RL,Nob Hill,200_fern alley,NaN,False
6108,Garbage/Recycle,S-HEIL,I,0,0,0,0,6,No,246,...,0,0,0,RGG,1,SL,Russian Hill,1455_filber,NaN,False
9796,Recycle,R-HEIL,I,0,0,0,0,7,Yes,981,...,0,0,0,RGG,1,RL,Nob Hill,1100_bush,NaN,False
9798,Recycle,R-HEIL,IC,0,0,0,3,3,No,981,...,0,0,0,RGG,1,RL,Nob Hill,1000_bush,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11707,Garbage/Compost,S-HEIL,I,0,0,0,0,2,NaN,231,...,0,0,0,RGG,0,SL,Financial District,1436_california,AOKEY,True
5376,Garbage/Recycle,S-HEIL,C,1,0,0,1,0,No,243,...,0,0,0,RGG,0,SL,Presidio,1310_kobb,NaN,False
11692,Garbage/Compost,S-HEIL,I,0,0,0,0,1,NaN,231,...,0,0,0,RGG,0,SL,Financial District,1353_bush,CCKEY,True
11729,Garbage/Compost,S-HEIL,I,0,0,0,0,1,NaN,231,...,0,0,0,RGG,0,SL,Financial District,1010_hyd,NaN,False


In [ ]:
final_data['2 yrd'].value_counts()

0    11533
1        7
2        2
3        1
Name: 2 yrd, dtype: int64

In [ ]:
data.columns

Index(['Commodity', 'Vehicle Type', 'Inside/Curb', '16 gal', '20 gal',
       '32 gal', '64 gal', '96 gal', 'Meandor', 'Route', 'Tipper', 'Day',
       '#Units', 'Time', '1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
       '5 yrd', '6 yrd', 'Company', 'Outlier', 'Truck', 'Neighborhood',
       'Address_Street', 'Code', 'Has Key'],
      dtype='object')

In [ ]:
data=final_data.loc[(rgg_rss['Neighborhood']=='Marina')]
data

,Commodity,Vehicle Type,Inside/Curb,16 gal,20 gal,32 gal,64 gal,96 gal,Meandor,Route,...,4 yrd,5 yrd,6 yrd,Company,Outlier,Truck,Neighborhood,Address_Street,Code,Has Key
6306,Recycle,S-LODAL,I,0,0,0,0,1,No,990,...,0,0,0,RGG,0,SL,Marina,3450_ierc,NaN,False
6307,Recycle,S-LODAL,I,0,0,0,0,1,No,990,...,0,0,0,RGG,0,SL,Marina,75_capra way,AAKEY,True
6308,Recycle,S-LODAL,I,0,0,0,0,1,No,990,...,0,0,0,RGG,0,SL,Marina,50_capra way,AAKEY,True
6309,Recycle,S-LODAL,C,0,0,0,2,0,No,990,...,0,0,0,RGG,0,SL,Marina,3514_ierc,NaN,False
6310,Recycle,S-LODAL,C,0,0,1,1,0,No,990,...,0,0,0,RGG,0,SL,Marina,3526_ierc,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11970,Garbage,R-HEIL,I,0,0,0,0,1,NaN,216,...,0,0,0,RGG,0,SL,Marina,1624_ejo,NaN,False
11978,Garbage,R-HEIL,I,0,0,0,0,1,Yes,216,...,0,0,0,RGG,0,SL,Marina,1566_green,NaN,False
11979,Garbage,R-HEIL,I,0,0,0,0,1,NaN,216,...,0,0,0,RGG,0,SL,Marina,1560_ejo,NaN,False
11982,Garbage,R-HEIL,I,0,0,0,0,1,NaN,216,...,0,0,0,RGG,0,SL,Marina,9366_n ness,NaN,False


In [ ]:
from imblearn.pipeline import Pipeline

def Prediction(Neighborhood,Test_X):

        data=final_data.loc[(rgg_rss['Neighborhood']==Neighborhood)]
        X=data[['Commodity', '16 gal', '20 gal','32 gal', '64 gal', '96 gal', '1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
        '5 yrd', '6 yrd',
                '#Units','Day', 'Company','Has Key']]
        y=data['Time']

        #X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state=16)

        def categorical_col_transform(X):
                X_c=X.copy()
                X_c['concat']=(X_c['16 gal'].map(str))+'_'+(X_c['20 gal'].map(str))+(X_c['32 gal'].map(str))+'_'
                +(X_c['64 gal']. map(str)) +(X_c['96 gal'].map(str))
                return X_c[['concat','Commodity','Day','Company','#Units','Has Key']]


        transformer = QuantileTransformer(output_distribution='normal',n_quantiles=250)
        regressor = LinearRegression()
        transformed_target_regr = TransformedTargetRegressor(regressor=regressor,
                                                        transformer=transformer)

        preprocessing = ColumnTransformer([
                                ('scaler',StandardScaler(),[1,2,3,4,5,6,7,8,9,10,11,12,13]),                       
                                ('categorical', OneHotEncoder(handle_unknown='ignore'), [0,14,15,16])
                                ])

        pipe = Pipeline([
                        #('col_selection',FunctionTransformer(categorical_col_transform,validate=False)),
                        ('imputation',SimpleImputer(missing_values=np.nan,strategy="most_frequent")),
                        ('preprocessing',preprocessing),
                        ('kbest',SelectKBest(mutual_info_regression, k=8)),
                        ('lr', RidgeCV())])

        pipe.fit(X, y)
        y_pred = pipe.predict(Test_X)
        #mse = mean_squared_error(y_validation, y_pred)
        #mae=mean_absolute_error(y_validation,y_pred)

        return y_pred


#print(mae,mse,"\n")
#print("\n".join("{} {}".format(x, y) for x, y in zip(y_pred, y_validation)))

In [ ]:
test_data=pd.read_excel('/work/Routes975_998-990_New.xlsx')

In [ ]:
test_data.columns

Index(['Z1SVC#', 'Commodity', '16 gal', '20 gal', '32 gal', '64 gal', '96 gal',
       '1 yd', '1.5 yd', '2 yd', '3 yd', '4 yd', '5 yd', '6 yd', '#Units',
       'Day', 'Company', 'Has Key', 'Neighborhood'],
      dtype='object')

In [ ]:
y_pred=Prediction('Marina',test_data[['Commodity',
        '16 gal', '20 gal','32 gal',  '64 gal', '96 gal', '1 yd', '1.5 yd', '2 yd','3 yd','4 yd', '5 yd', '6 yd','#Units','Day', 'Company', 'Has Key']])

In [ ]:
y_pred

array([74.5412503 , 56.03811408, 56.03811408, ..., 74.5412503 ,
       56.03811408, 56.03811408])

In [ ]:
test_data['Predicted Time'] = y_pred
test_data.to_excel('Routes975_998-990_Predicted_Times_Marina_new.xlsx',index=False)


In [ ]:
X=data[['Commodity', '16 gal', '20 gal','32 gal', '64 gal', '96 gal',
        '#Units','Day', 'Company','Has Key','Truck']]
X

,Commodity,16 gal,20 gal,32 gal,64 gal,96 gal,#Units,Day,Company,Has Key,Truck
0,Recycle,0,0,1,2,2,5,2,RSS,False,SL
1,Recycle,0,0,0,0,1,1,2,RSS,False,SL
2,Recycle,0,0,0,0,1,1,2,RSS,False,SL
3,Recycle,0,0,1,0,1,2,2,RSS,False,SL
4,Recycle,0,0,1,0,0,1,2,RSS,False,SL
...,...,...,...,...,...,...,...,...,...,...,...
4246,Garbage,2,0,3,0,0,5,2,RSS,False,NaN
4247,Garbage,2,0,3,1,0,6,2,RSS,False,NaN
4248,Garbage,0,2,4,0,0,6,2,RSS,False,NaN
4249,Garbage,1,1,4,0,0,6,2,RSS,False,NaN


## Deep dive into Inner Richmond

In [ ]:
import seaborn as sns

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597 entries, 0 to 4250
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Commodity       597 non-null    object 
 1   Vehicle Type    597 non-null    object 
 2   Inside/Curb     597 non-null    object 
 3   16 gal          597 non-null    int64  
 4   20 gal          597 non-null    int64  
 5   32 gal          597 non-null    int64  
 6   64 gal          597 non-null    int64  
 7   96 gal          597 non-null    int64  
 8   Meandor         2 non-null      object 
 9   Route           597 non-null    int64  
 10  Tipper          597 non-null    object 
 11  Day             597 non-null    object 
 12  #Units          597 non-null    int64  
 13  Time            597 non-null    float64
 14  Company         597 non-null    object 
 15  Outlier         597 non-null    int64  
 16  Truck           348 non-null    object 
 17  Neighborhood    597 non-null    ob

In [ ]:
data=rgg_rss.loc[rgg_rss['Neighborhood']=='Inner Richmond']
data['16 gal']=data['16 gal'].map(str) 
data['concat']=(data['16 gal'].map(str) )+'_'+data['20 gal'].map(str)+'_'+data['32 gal'].map(str)+'_'+data['64 gal'].map(str) +'_'+data['96 gal'].map(str)
data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Commodity,Vehicle Type,Inside/Curb,16 gal,20 gal,32 gal,64 gal,96 gal,Meandor,Route,...,#Units,Time,Company,Outlier,Truck,Neighborhood,Address_Street,Code,Has Key,concat
0,Recycle,S-HEIL,C,0,0,1,2,2,NaN,912,...,5,118.0,RSS,0,SL,Inner Richmond,5128_geary,NaN,False,0_0_1_2_2
1,Recycle,S-HEIL,C,0,0,0,0,1,NaN,912,...,1,59.0,RSS,0,SL,Inner Richmond,5620_geary,NaN,False,0_0_0_0_1
2,Recycle,S-HEIL,I,0,0,0,0,1,NaN,912,...,1,86.0,RSS,0,SL,Inner Richmond,1947_clement,NaN,False,0_0_0_0_1
3,Recycle,S-HEIL,C,0,0,1,0,1,NaN,912,...,2,41.0,RSS,0,SL,Inner Richmond,1919_clement,NaN,False,0_0_1_0_1
4,Recycle,S-HEIL,C,0,0,1,0,0,NaN,912,...,1,31.0,RSS,0,SL,Inner Richmond,1909_clement,NaN,False,0_0_1_0_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4246,Garbage,na,C,2,0,3,0,0,NaN,9,...,5,125.0,RSS,0,NaN,Inner Richmond,NaN,NaN,False,2_0_3_0_0
4247,Garbage,na,C,2,0,3,1,0,NaN,9,...,6,98.0,RSS,0,NaN,Inner Richmond,NaN,NaN,False,2_0_3_1_0
4248,Garbage,na,C,0,2,4,0,0,NaN,9,...,6,97.0,RSS,0,NaN,Inner Richmond,NaN,NaN,False,0_2_4_0_0
4249,Garbage,na,C,1,1,4,0,0,NaN,9,...,6,99.0,RSS,0,NaN,Inner Richmond,NaN,NaN,False,1_1_4_0_0


In [ ]:
data[['Time','concat']].sort_values(by=['Time'],ascending=False)

,Time,concat
3007,1428.0,0_0_3_4_0
3016,845.0,0_0_2_4_0
23,323.0,0_0_0_1_2
3012,309.0,0_0_1_1_1
211,296.0,0_0_3_3_1
...,...,...
4217,19.0,0_1_0_0_0
3246,19.0,0_0_0_1_0
2177,18.0,0_0_0_1_0
2148,17.0,0_0_1_0_0


In [ ]:
data[['Time','concat']].value_counts()

Time    concat   
51.0    0_0_0_2_0    5
38.0    0_0_2_0_0    5
37.0    1_0_1_0_0    4
27.0    0_0_1_0_0    4
42.0    0_0_0_2_0    4
                    ..
60.0    0_0_1_1_1    1
        0_0_0_0_1    1
59.0    2_0_1_0_0    1
        0_1_3_0_0    1
1428.0  0_0_3_4_0    1
Length: 475, dtype: int64

In [ ]:
data.loc[data['Time']==1428.0]

,Commodity,Vehicle Type,Inside/Curb,16 gal,20 gal,32 gal,64 gal,96 gal,Meandor,Route,...,Day,#Units,Time,Company,Outlier,Truck,Neighborhood,Address_Street,Code,Has Key
3007,Garbage,na,C,0,0,3,4,0,NaN,9,...,2,7,1428.0,RSS,1,NaN,Inner Richmond,NaN,NaN,False


In [ ]:
data.loc[data['Time']==1428.0]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=938c6ad9-491d-4307-bf8a-c751a244ce4f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>